In [1]:
pip install ucimlrepo

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import tensorflow as tf
from ucimlrepo import fetch_ucirepo

# Fetching the dataset
student_performance = fetch_ucirepo(id=320)

# Extracting features and targets
X = student_performance.data.features
y = student_performance.data.targets

# Converting features and targets to Pandas DataFrames
X_df = pd.DataFrame(X, columns=student_performance.metadata.features)
y_df = pd.DataFrame(y, columns=student_performance.metadata.targets)

# Concatenating features and targets into a single DataFrame
student_df = pd.concat([X_df, y_df], axis=1)

In [3]:
# Creating pass/fail category for 'G3'
# True ('pass' or 1) if >= 10, else False ('fail' or 0)
student_df["G3"] = student_df["G3"].astype(int)
student_df["pass_fail"] = (student_df["G3"] >= 10).astype(int)

In [4]:
# Dropping original 'G3'
student_df = student_df.drop(columns = "G3")

# Encoding

In [5]:
# Converting categorical data to numeric with 'pd.get_dummies'
student_df = pd.get_dummies(student_df)

In [6]:
# Splitting preprocessed data into features and target arrays
X = student_df.drop(columns = "pass_fail", axis = 1)
y= student_df["pass_fail"]

# Splitting the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

# Scaling

In [7]:
# Creating a StandardScaler instance
scaler = StandardScaler()

# Fitting the StandardScaler
X_scaler = scaler.fit(X_train)

# Scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train, and Evaluate the Model

In [8]:
# Defining the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# Optimized hyperparameters based on Chalisa's educated trial-and-error work
features = X_train_scaled.shape[1]
nodes_layer1 = 100
nodes_layer2 = 50
nodes_layer3 = 30
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = nodes_layer1, input_dim = features, activation = "relu"))
nn.add(tf.keras.layers.Dropout(0.4))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = nodes_layer2, activation = "relu"))
nn.add(tf.keras.layers.Dropout(0.4))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = nodes_layer3, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Checking the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               5900      
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 30)                1530      
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 12511 (48.87 KB)
Trainable params: 12511 (

In [9]:
# Compiling the model
nn.compile(loss = "binary_crossentropy", optimizer = "sgd", metrics = ["accuracy"])

In [10]:
# Training the model
fitted_model = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
16/16 [==============================] - 1s 3ms/step - loss: 0.6228 - accuracy: 0.6852
Epoch 2/100
16/16 [==============================] - 0s 2ms/step - loss: 0.5796 - accuracy: 0.7901
Epoch 3/100
16/16 [==============================] - 0s 3ms/step - loss: 0.5055 - accuracy: 0.8251
Epoch 4/100
16/16 [==============================] - 0s 3ms/step - loss: 0.5181 - accuracy: 0.8313
Epoch 5/100
16/16 [==============================] - 0s 3ms/step - loss: 0.4833 - accuracy: 0.8374
Epoch 6/100
16/16 [==============================] - 0s 3ms/step - loss: 0.4606 - accuracy: 0.8477
Epoch 7/100
16/16 [==============================] - 0s 3ms/step - loss: 0.4609 - accuracy: 0.8519
Epoch 8/100
16/16 [==============================] - 0s 2ms/step - loss: 0.4601 - accuracy: 0.8436
Epoch 9/100
16/16 [==============================] - 0s 2ms/step - loss: 0.4440 - accuracy: 0.8498
Epoch 10/100
16/16 [==============================] - 0s 3ms/step - loss: 0.4362 - accuracy: 0.8477
Epoch 11/

In [11]:
# Evaluating the model using training data
model_loss, model_accuracy = nn.evaluate(X_train_scaled,y_train,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

16/16 - 0s - loss: 0.1575 - accuracy: 0.9527 - 188ms/epoch - 12ms/step
Loss: 0.15746255218982697, Accuracy: 0.952674925327301


In [12]:
# Evaluating the model using test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6/6 - 0s - loss: 0.2713 - accuracy: 0.8896 - 56ms/epoch - 9ms/step
Loss: 0.2713446021080017, Accuracy: 0.8895705342292786


## [Permutation Feature Importance](https://scikit-learn.org/stable/modules/permutation_importance.html)

### R<sup>2</sup> (coefficient of determination) regression score function [r2_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html#sklearn.metrics.r2_score)

In [13]:
from sklearn.inspection import permutation_importance

# Loading the model
model_for_pfi = nn

# Loading the dataset
X_for_pfi = X_test_scaled
y_for_pfi = y_test

# Computing permutation feature importance
pfi_result = permutation_importance(model_for_pfi, X_for_pfi, y_for_pfi, scoring = 'r2', n_repeats = 10, random_state = 1)

# Printing feature importances in descending order
feature_names = student_df.drop(['pass_fail'], axis = 1).columns.tolist()
for i in pfi_result.importances_mean.argsort()[::-1]:
    print(f"{feature_names[i]}: {pfi_result.importances_mean[i]:.3f} ± {pfi_result.importances_std[i]:.3f}")


6/6 [==============================] - 0s 3ms/step
G2: 0.154 ± 0.042
G1: 0.087 ± 0.025
failures: 0.039 ± 0.014
higher_no: 0.023 ± 0.008
paid_yes: 0.018 ± 0.009
reason_home: 0.013 ± 0.010
schoolsup_no: 0.013 ± 0.008
famrel: 0.011 ± 0.008
Dalc: 0.011 ± 0.008
Pstatus_T: 0.009 ± 0.007
absences: 0.008 ± 0.007
higher_yes: 0.006 ± 0.007
guardian_other: 0.005 ± 0.004
Mjob_teacher: 0.005 ± 0.006
Fjob_at_home: 0.005 ± 0.007
romantic_yes: 0.005 ± 0.007
reason_other: 0.005 ± 0.010
Medu: 0.005 ± 0.007
traveltime: 0.004 ± 0.005
Walc: 0.003 ± 0.005
Mjob_health: 0.003 ± 0.005
Fjob_teacher: 0.003 ± 0.006
studytime: 0.003 ± 0.006
school_MS: 0.001 ± 0.012
Fjob_health: 0.000 ± 0.001
Pstatus_A: 0.000 ± 0.005
schoolsup_yes: -0.001 ± 0.007
freetime: -0.002 ± 0.005
address_U: -0.002 ± 0.007
nursery_yes: -0.002 ± 0.006
age: -0.002 ± 0.005
Fedu: -0.003 ± 0.005
reason_course: -0.004 ± 0.006
romantic_no: -0.004 ± 0.006
activities_yes: -0.004 ± 0.007
school_GP: -0.005 ± 0.013
internet_yes: -0.005 ± 0.005
paid_no: 

In [14]:
# Printing only the top ten feature importances
sorted_pfi_result_desc = pfi_result.importances_mean.argsort()[::-1]
top_ten_sorted_pfi_result_desc = sorted_pfi_result_desc[:10]

for i in top_ten_sorted_pfi_result_desc:
    print(f"{feature_names[i]}: {pfi_result.importances_mean[i]:.3f} ± {pfi_result.importances_std[i]:.3f}")

G2: 0.154 ± 0.042
G1: 0.087 ± 0.025
failures: 0.039 ± 0.014
higher_no: 0.023 ± 0.008
paid_yes: 0.018 ± 0.009
reason_home: 0.013 ± 0.010
schoolsup_no: 0.013 ± 0.008
famrel: 0.011 ± 0.008
Dalc: 0.011 ± 0.008
Pstatus_T: 0.009 ± 0.007


### Explained variance regression score function [explained_variance_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.explained_variance_score.html#sklearn.metrics.explained_variance_score)

In [15]:
# Computing permutation feature importance
pfi_result_2 = permutation_importance(model_for_pfi, X_for_pfi, y_for_pfi, scoring = 'explained_variance', n_repeats = 10, random_state = 1)

# Printing feature importances in descending order
for i in pfi_result_2.importances_mean.argsort()[::-1]:
    print(f"{feature_names[i]}: {pfi_result_2.importances_mean[i]:.3f} ± {pfi_result_2.importances_std[i]:.3f}")

6/6 [==============================] - 0s 3ms/step
G2: 0.133 ± 0.043
G1: 0.069 ± 0.025
failures: 0.035 ± 0.014
higher_no: 0.021 ± 0.008
paid_yes: 0.017 ± 0.009
schoolsup_no: 0.013 ± 0.008
reason_home: 0.012 ± 0.010
famrel: 0.011 ± 0.008
Dalc: 0.009 ± 0.008
Pstatus_T: 0.008 ± 0.007
absences: 0.006 ± 0.007
romantic_yes: 0.006 ± 0.007
guardian_other: 0.005 ± 0.004
higher_yes: 0.005 ± 0.007
Mjob_teacher: 0.005 ± 0.005
Fjob_at_home: 0.005 ± 0.008
Medu: 0.004 ± 0.007
reason_other: 0.004 ± 0.009
Walc: 0.003 ± 0.005
traveltime: 0.003 ± 0.005
Mjob_health: 0.003 ± 0.005
Fjob_teacher: 0.002 ± 0.007
studytime: 0.001 ± 0.006
Fjob_health: 0.000 ± 0.001
schoolsup_yes: -0.001 ± 0.007
Pstatus_A: -0.001 ± 0.005
address_U: -0.001 ± 0.007
freetime: -0.002 ± 0.005
nursery_yes: -0.002 ± 0.006
Fedu: -0.003 ± 0.005
age: -0.004 ± 0.005
school_MS: -0.004 ± 0.013
romantic_no: -0.004 ± 0.006
reason_course: -0.004 ± 0.006
internet_yes: -0.005 ± 0.006
activities_yes: -0.005 ± 0.006
paid_no: -0.006 ± 0.006
Mjob_at_h

In [16]:
# Printing only the top ten feature importances
sorted_pfi_result_2_desc = pfi_result_2.importances_mean.argsort()[::-1]
top_ten_sorted_pfi_result_2_desc = sorted_pfi_result_2_desc[:10]

for i in top_ten_sorted_pfi_result_2_desc:
    print(f"{feature_names[i]}: {pfi_result_2.importances_mean[i]:.3f} ± {pfi_result_2.importances_std[i]:.3f}")

G2: 0.133 ± 0.043
G1: 0.069 ± 0.025
failures: 0.035 ± 0.014
higher_no: 0.021 ± 0.008
paid_yes: 0.017 ± 0.009
schoolsup_no: 0.013 ± 0.008
reason_home: 0.012 ± 0.010
famrel: 0.011 ± 0.008
Dalc: 0.009 ± 0.008
Pstatus_T: 0.008 ± 0.007


### [Using multiple scorers](https://scikit-learn.org/stable/modules/permutation_importance.html#:~:text=below%20we%20use-,a%20list%20of%20metrics,-%2C%20but%20more%20input)

In [21]:
scoring = ['max_error',
           'neg_mean_absolute_error',
           'neg_mean_squared_error',
           'neg_root_mean_squared_error',
           'neg_mean_squared_log_error',
           'neg_median_absolute_error',
           'neg_mean_absolute_percentage_error']
r_multi = permutation_importance(model_for_pfi, X_for_pfi, y_for_pfi, n_repeats = 10, random_state = 1, scoring = scoring)

for metric in r_multi:
    print(f"{metric}")
    r = r_multi[metric]
    for i in r.importances_mean.argsort()[::-1]:
        if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
            print(f"    {feature_names[i]:<8}"
                  f"{r.importances_mean[i]:.3f}"
                  f" +/- {r.importances_std[i]:.3f}")

6/6 [==============================] - 0s 3ms/step
max_error
    G2      0.030 +/- 0.005
    G1      0.024 +/- 0.008
    failures0.021 +/- 0.000
    Medu    0.015 +/- 0.004
    address_R0.012 +/- 0.003
neg_mean_absolute_error
    G2      0.015 +/- 0.007
    failures0.005 +/- 0.002
    higher_no0.004 +/- 0.001
neg_mean_squared_error
    G2      0.021 +/- 0.006
    G1      0.012 +/- 0.003
    failures0.005 +/- 0.002
    higher_no0.003 +/- 0.001
    paid_yes0.002 +/- 0.001
neg_root_mean_squared_error
    G2      0.034 +/- 0.009
    G1      0.020 +/- 0.005
    failures0.009 +/- 0.003
    higher_no0.005 +/- 0.002
    paid_yes0.004 +/- 0.002
neg_mean_squared_log_error
    G2      0.011 +/- 0.002
    G1      0.007 +/- 0.001
    failures0.003 +/- 0.001
    higher_no0.002 +/- 0.000
    paid_yes0.001 +/- 0.001
neg_median_absolute_error
    G2      0.006 +/- 0.002
neg_mean_absolute_percentage_error
    G2      93251724304132.703 +/- 14189708099085.371
    G1      66660037735500.648 +/- 9273799666